In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x132f868c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x132fb4ac0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
"""
1)Messages are the building blocks of a conversation with an LLM.
2)They are used to represent the input and output of a chat model, 
as well as any additional context or metadata 
that may be associated with a conversation.

3)Each message has a role (e.g., "user", "assistant") 
and content (e.g., text, multimodal data) 
with additional metadata that varies 
depending on the chat model provider.
"""

from langchain_core.messages import HumanMessage, SystemMessage

#System Message ---> Tells system on how to behave
#Human Message ---> User input
#AI Message ---> Represents the LLM's response
messages = [
    SystemMessage(content="You are a cricket expert"),
    HumanMessage(content="Which cricketer has scored the most number of runs in ODI's?"),
]

gemma_response = model.invoke(messages)

gemma_response

AIMessage(content='That title belongs to **Rohit Sharma** from India! \n\nHe surpassed the great Sachin Tendulkar in 2023 to become the highest run-scorer in ODI history. 🏏  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 28, 'total_tokens': 74, 'completion_time': 0.083636364, 'prompt_time': 0.002153561, 'queue_time': 0.2418039, 'total_time': 0.085789925}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--370f3e1a-69e6-4b83-add2-94c4a469ce8b-0', usage_metadata={'input_tokens': 28, 'output_tokens': 46, 'total_tokens': 74})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(gemma_response.content)
# The response is a string, so we can use it directly
# or parse it further if needed
print(gemma_response.content)

That title belongs to **Rohit Sharma** from India! 

He surpassed the great Sachin Tendulkar in 2023 to become the highest run-scorer in ODI history. 🏏  




In [ ]:
#Using LCEL - chain the components

chain = model | parser
chain.invoke(messages)

"That would be **Rohit Sharma** from India!  \n\nHe surpassed Sachin Tendulkar's long-standing record in December 2022.  \n\n"

In [ ]:
## Prompt Templates

from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following text to French: {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [11]:
language_response = prompt.invoke({"language": "English", "text": "Hello, how are you?"})

language_response.to_messages()

[SystemMessage(content='Translate the following text to French: English', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [ ]:
#Chaining components together using LCEL

#We created 3 components
# 1. Prompt Template
# 2. Model
# 3. Output Parser

chain = prompt | model | parser
chain.invoke({"language": "English", "text": "Hello, how are you?"})

"French: Bonjour, comment allez-vous ? \n\n\nLet me know if you have anything else you'd like me to translate!\n"